In [1]:
from keras.models import Sequential , Model
from keras.layers import Dense , Conv2D , MaxPool2D , BatchNormalization , Reshape , Input , Flatten
from keras.layers import UpSampling2D
from keras.layers.advanced_activations import LeakyReLU

from keras.optimizers import Adam

from keras.datasets import mnist

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os

import matplotlib as plt
import numpy as np

import gc

%matplotlib inline

In [3]:
WIDTH = 96
HEIGHT = 96
CHANNEL = 3

LATENT_DIM = 100 #latent variable z

BATCH_SIZE = 64
EPOCHS = 10

PATH = 'faces/'

#生成多少个图像 长*宽
ROW = 5
COL = 5

In [4]:
load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)

In [5]:
'''
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)
'''

'\n(X_train , y_train),(X_test , y_test) = mnist.load_data()\nX_train = X_train/127.5-1\nX_train = np.expand_dims(X_train , 3)\n'

In [6]:
'''
def load_mnist():
    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('images/No.%d.png' % epoch)
    plt.pyplot.close()

'''

"\ndef load_mnist():\n    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]\n    \ndef write_image_mnist(epoch):\n    \n    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))\n    generated_image = generator_i.predict(noise)\n    generated_image = generated_image*0.5+0.5\n    \n    fig , axes = plt.pyplot.subplots(ROW , COL)\n    \n    count=0\n    \n    for i in range(ROW):\n        for j in range(COL):\n            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')\n            axes[i][j].axis('off')\n            count += 1\n            \n    fig.savefig('images/No.%d.png' % epoch)\n    plt.pyplot.close()\n\n"

In [7]:
def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return np.array(images)/127.5-1

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = (generated_image+1)*127.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('generated_faces_naive_gans/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')

In [8]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    #model.add(Dense(units = 128 , input_dim = LATENT_DIM , name='dense0'))
    #model.add(LeakyReLU(0.2))
    #model.add(BatchNormalization(momentum=0.8 , name='batchnorm1'))
    model.add(Dense(units = 256 , input_dim = LATENT_DIM , name='dense1'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm1'))
    model.add(Dense(units = 512 , name='dense2'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm2'))
    model.add(Dense(units = 1024 , name='dense3'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm3'))
    #model.add(Dense(units = 5000 , name='dense4'))
    #model.add(LeakyReLU(0.2))
    #model.add(BatchNormalization(momentum=0.8 , name='batchnorm4'))
    #model.add(Dense(units = 10000 , name='dense5'))
    #model.add(LeakyReLU(0.2))
    #model.add(BatchNormalization(momentum=0.8 , name='batchnorm5'))
    #model.add(Dense(units = 20000 , activation='relu' , name='dense6'))
    #model.add(BatchNormalization(name='batchnorm6'))
    model.add(Dense(units = WIDTH*HEIGHT*CHANNEL , activation='tanh' , name='dense7'))
    model.add(Reshape(target_shape=(WIDTH , HEIGHT , CHANNEL) , name='reshape1'))
    
    #model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    image = model(noise)
    
    return Model(noise , image , name='generator_Model')

In [9]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Flatten(input_shape = (WIDTH,HEIGHT,CHANNEL) , name='flatten1'))
    #model.add(Dense(units=10000 , name='dense8'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu1'))
    #model.add(Dense(units=5000 , name='dense9'))
    #model.add(LeakyReLU(0.2 , name='leakyrelu2'))
    #model.add(Dense(units=1024 , name='dense10'))
    #model.add(LeakyReLU(0.2 , name='leakyrelu3'))
    model.add(Dense(units=512 , name='dense11'))
    model.add(LeakyReLU(0.2 , name='leakyrelu4'))
    model.add(Dense(units=256 , name='dense12'))
    model.add(LeakyReLU(0.2 , name='leakyrelu5'))
    model.add(Dense(units=128 , name='dense13'))
    model.add(LeakyReLU(0.2 , name='leakyrelu6'))
    #model.add(Dense(units=32 , name='dense14'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu7'))
    #model.add(Dense(units=4 , name='dense15'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu8'))
    model.add(Dense(units=1 , activation='sigmoid' , name='dense16'))
    
    #model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    validity = model(image)
    
    return Model(image , validity , name='discriminator_Model')

In [10]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    
    image = generator_i(z)
    discriminator_i.trainable = False
    validity = discriminator_i(image)
    
    return Model(z , validity , name='combined_model')

In [11]:
adam = Adam(lr = 0.0002 , beta_1=0.5)

In [12]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=adam , loss='binary_crossentropy' , metrics=['accuracy'])

generator_i = generator()

combined_model_i = combined_model(generator_i , discriminator_i)


combined_model_i.compile(optimizer=adam , loss='binary_crossentropy')

In [ ]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(20000):
    noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

    real_image = load_image()
    #real_image = load_mnist()
    #训练判别器
    fake_image = generator_i.predict(noise)

    real_loss = discriminator_i.train_on_batch(real_image , real_labels)
    fake_loss = discriminator_i.train_on_batch(fake_image , fake_labels)

    loss = np.add(real_loss , fake_loss)/2

    #训练生成器
    noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    generator_loss = combined_model_i.train_on_batch(noise2 , real_labels)
    
    print('epoch:%d loss:%f accu:%f gene_loss:%f' % (i , loss[0] , loss[1] , generator_loss))
    
    if i % 100 == 0:
        write_image(i)
        #write_image_mnist(i)
    
write_image(999)

C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 loss:0.759656 accu:0.539062 gene_loss:0.688720
epoch:1 loss:0.696805 accu:0.523438 gene_loss:0.604391
epoch:2 loss:0.539400 accu:0.554688 gene_loss:0.552086
epoch:3 loss:0.572074 accu:0.585938 gene_loss:0.590204
epoch:4 loss:0.601807 accu:0.570312 gene_loss:0.622905
epoch:5 loss:0.533121 accu:0.609375 gene_loss:0.783461
epoch:6 loss:0.550447 accu:0.640625 gene_loss:0.906205
epoch:7 loss:0.367006 accu:0.789062 gene_loss:1.192909
epoch:8 loss:0.360047 accu:0.789062 gene_loss:1.310358
epoch:9 loss:0.309995 accu:0.828125 gene_loss:1.558204
epoch:10 loss:0.321382 accu:0.835938 gene_loss:1.838410
epoch:11 loss:0.182473 accu:0.945312 gene_loss:2.000021
epoch:12 loss:0.155639 accu:0.960938 gene_loss:2.191169
epoch:13 loss:0.138086 accu:0.968750 gene_loss:2.326557
epoch:14 loss:0.154411 accu:0.968750 gene_loss:2.656355
epoch:15 loss:0.110758 accu:0.992188 gene_loss:2.651098
epoch:16 loss:0.380610 accu:0.835938 gene_loss:2.674984
epoch:17 loss:0.738036 accu:0.820312 gene_loss:2.903328
ep

epoch:146 loss:0.235849 accu:0.898438 gene_loss:2.005168
epoch:147 loss:0.386593 accu:0.820312 gene_loss:2.167719
epoch:148 loss:0.209133 accu:0.906250 gene_loss:2.527414
epoch:149 loss:0.291685 accu:0.898438 gene_loss:2.103581
epoch:150 loss:0.303907 accu:0.835938 gene_loss:2.639211
epoch:151 loss:0.148308 accu:0.984375 gene_loss:2.436734
epoch:152 loss:0.250531 accu:0.906250 gene_loss:2.373302
epoch:153 loss:0.206862 accu:0.929688 gene_loss:2.297334
epoch:154 loss:0.210256 accu:0.914062 gene_loss:2.509369
epoch:155 loss:0.254071 accu:0.890625 gene_loss:2.116045
epoch:156 loss:0.242136 accu:0.890625 gene_loss:3.035667
epoch:157 loss:0.315196 accu:0.906250 gene_loss:2.277182
epoch:158 loss:0.348751 accu:0.843750 gene_loss:2.570379
epoch:159 loss:0.408110 accu:0.789062 gene_loss:2.332411
epoch:160 loss:0.266034 accu:0.843750 gene_loss:2.731567
epoch:161 loss:0.228674 accu:0.929688 gene_loss:3.103270
epoch:162 loss:0.667269 accu:0.734375 gene_loss:3.275549
epoch:163 loss:0.698309 accu:0.

epoch:290 loss:0.584306 accu:0.804688 gene_loss:5.603207
epoch:291 loss:1.092807 accu:0.531250 gene_loss:2.830513
epoch:292 loss:0.780769 accu:0.750000 gene_loss:6.202095
epoch:293 loss:0.394484 accu:0.765625 gene_loss:3.651851
epoch:294 loss:0.310028 accu:0.843750 gene_loss:4.630879
epoch:295 loss:0.422092 accu:0.867188 gene_loss:3.539567
epoch:296 loss:0.268562 accu:0.921875 gene_loss:4.390516
epoch:297 loss:0.557962 accu:0.734375 gene_loss:3.146330
epoch:298 loss:0.270708 accu:0.898438 gene_loss:4.637569
epoch:299 loss:0.446894 accu:0.765625 gene_loss:2.883954
epoch:300 loss:0.187973 accu:0.929688 gene_loss:3.356745
epoch:301 loss:0.294904 accu:0.867188 gene_loss:2.543774
epoch:302 loss:0.391522 accu:0.835938 gene_loss:3.289855
epoch:303 loss:0.310474 accu:0.843750 gene_loss:2.757254
epoch:304 loss:0.546996 accu:0.695312 gene_loss:3.824794
epoch:305 loss:0.466500 accu:0.742188 gene_loss:3.538920
epoch:306 loss:0.350452 accu:0.851562 gene_loss:3.087962
epoch:307 loss:0.507528 accu:0.

epoch:434 loss:0.512788 accu:0.742188 gene_loss:2.190266
epoch:435 loss:0.486591 accu:0.804688 gene_loss:2.054175
epoch:436 loss:0.533399 accu:0.750000 gene_loss:1.733258
epoch:437 loss:0.496947 accu:0.750000 gene_loss:2.026785
epoch:438 loss:0.446279 accu:0.812500 gene_loss:2.694504
epoch:439 loss:0.696446 accu:0.593750 gene_loss:1.947655
epoch:440 loss:0.494824 accu:0.734375 gene_loss:1.961428
epoch:441 loss:0.495780 accu:0.750000 gene_loss:2.237188
epoch:442 loss:0.424724 accu:0.835938 gene_loss:2.823909
epoch:443 loss:0.491532 accu:0.718750 gene_loss:2.701417
epoch:444 loss:0.581059 accu:0.671875 gene_loss:2.628862
epoch:445 loss:0.360138 accu:0.851562 gene_loss:2.514719
epoch:446 loss:0.402522 accu:0.781250 gene_loss:2.951564
epoch:447 loss:0.471062 accu:0.765625 gene_loss:2.378960
epoch:448 loss:0.467758 accu:0.734375 gene_loss:3.078390
epoch:449 loss:0.465443 accu:0.757812 gene_loss:3.126143
epoch:450 loss:0.313090 accu:0.859375 gene_loss:2.995707
epoch:451 loss:0.426935 accu:0.

...

epoch:2289 loss:0.494589 accu:0.750000 gene_loss:3.443178
epoch:2290 loss:0.349461 accu:0.820312 gene_loss:2.850672
epoch:2291 loss:0.436303 accu:0.781250 gene_loss:2.804859
epoch:2292 loss:0.331254 accu:0.859375 gene_loss:3.022944
epoch:2293 loss:0.385743 accu:0.828125 gene_loss:3.469813
epoch:2294 loss:0.201373 accu:0.937500 gene_loss:3.388127
epoch:2295 loss:0.382227 accu:0.828125 gene_loss:3.198915
epoch:2296 loss:0.416610 accu:0.835938 gene_loss:3.875462
epoch:2297 loss:0.325317 accu:0.859375 gene_loss:3.983942
epoch:2298 loss:0.331399 accu:0.859375 gene_loss:4.694100
epoch:2299 loss:0.415093 accu:0.804688 gene_loss:3.833491
epoch:2300 loss:0.401317 accu:0.789062 gene_loss:2.998681
epoch:2301 loss:0.446559 accu:0.781250 gene_loss:4.242595
epoch:2302 loss:0.443765 accu:0.820312 gene_loss:3.336616
epoch:2303 loss:0.445075 accu:0.765625 gene_loss:3.053889
epoch:2304 loss:0.391171 accu:0.812500 gene_loss:3.317915
epoch:2305 loss:0.292903 accu:0.890625 gene_loss:3.458276
epoch:2306 los

epoch:2431 loss:0.335722 accu:0.851562 gene_loss:2.669684
epoch:2432 loss:0.371267 accu:0.835938 gene_loss:2.660990
epoch:2433 loss:0.367315 accu:0.820312 gene_loss:3.360471
epoch:2434 loss:0.358274 accu:0.835938 gene_loss:2.865497
epoch:2435 loss:0.328936 accu:0.875000 gene_loss:2.562691
epoch:2436 loss:0.355284 accu:0.828125 gene_loss:2.885232
epoch:2437 loss:0.269188 accu:0.882812 gene_loss:3.328188
epoch:2438 loss:0.343053 accu:0.851562 gene_loss:3.063449
epoch:2439 loss:0.264216 accu:0.906250 gene_loss:4.423669
epoch:2440 loss:0.353893 accu:0.859375 gene_loss:3.463738
epoch:2441 loss:0.312864 accu:0.890625 gene_loss:3.144564
epoch:2442 loss:0.338213 accu:0.835938 gene_loss:3.883231
epoch:2443 loss:0.426158 accu:0.781250 gene_loss:4.230938
epoch:2444 loss:0.459336 accu:0.804688 gene_loss:5.787640
epoch:2445 loss:0.867417 accu:0.640625 gene_loss:3.007196
epoch:2446 loss:0.341876 accu:0.851562 gene_loss:3.554371
epoch:2447 loss:0.378633 accu:0.820312 gene_loss:3.380493
epoch:2448 los

epoch:2573 loss:0.334464 accu:0.851562 gene_loss:2.551752
epoch:2574 loss:0.306977 accu:0.882812 gene_loss:2.772410
epoch:2575 loss:0.398267 accu:0.812500 gene_loss:3.183975
epoch:2576 loss:0.371805 accu:0.851562 gene_loss:2.661225
epoch:2577 loss:0.289353 accu:0.867188 gene_loss:3.204930
epoch:2578 loss:0.255521 accu:0.882812 gene_loss:2.710519
epoch:2579 loss:0.256692 accu:0.921875 gene_loss:2.752061
epoch:2580 loss:0.284770 accu:0.867188 gene_loss:3.833048
epoch:2581 loss:0.360194 accu:0.812500 gene_loss:3.244020
epoch:2582 loss:0.272395 accu:0.906250 gene_loss:2.932166
epoch:2583 loss:0.276578 accu:0.906250 gene_loss:3.138951
epoch:2584 loss:0.378870 accu:0.789062 gene_loss:4.162862
epoch:2585 loss:0.298038 accu:0.875000 gene_loss:4.690175
epoch:2586 loss:0.366251 accu:0.828125 gene_loss:3.199679
epoch:2587 loss:0.278485 accu:0.890625 gene_loss:3.089111
epoch:2588 loss:0.304022 accu:0.875000 gene_loss:3.517260
epoch:2589 loss:0.455730 accu:0.757812 gene_loss:4.723786
epoch:2590 los

epoch:2715 loss:0.580964 accu:0.773438 gene_loss:3.097131
epoch:2716 loss:0.582361 accu:0.687500 gene_loss:3.957942
epoch:2717 loss:0.520984 accu:0.781250 gene_loss:2.759219
epoch:2718 loss:0.349921 accu:0.835938 gene_loss:3.059858
epoch:2719 loss:0.438338 accu:0.843750 gene_loss:3.478764
epoch:2720 loss:0.335266 accu:0.843750 gene_loss:3.090614
epoch:2721 loss:0.358400 accu:0.859375 gene_loss:3.476018
epoch:2722 loss:0.373121 accu:0.851562 gene_loss:3.264416
epoch:2723 loss:0.335933 accu:0.828125 gene_loss:3.114653
epoch:2724 loss:0.428421 accu:0.781250 gene_loss:3.276283
epoch:2725 loss:0.529827 accu:0.804688 gene_loss:3.279883
epoch:2726 loss:0.417334 accu:0.804688 gene_loss:3.223599
epoch:2727 loss:0.340924 accu:0.859375 gene_loss:2.357323
epoch:2728 loss:0.301272 accu:0.875000 gene_loss:2.820442
epoch:2729 loss:0.323019 accu:0.851562 gene_loss:3.867001
epoch:2730 loss:0.290836 accu:0.890625 gene_loss:2.806108
epoch:2731 loss:0.267397 accu:0.890625 gene_loss:4.057600
epoch:2732 los

epoch:2857 loss:0.552298 accu:0.734375 gene_loss:2.731530
epoch:2858 loss:0.454569 accu:0.828125 gene_loss:2.741940
epoch:2859 loss:0.358143 accu:0.828125 gene_loss:2.418973
epoch:2860 loss:0.338725 accu:0.859375 gene_loss:2.778499
epoch:2861 loss:0.327863 accu:0.851562 gene_loss:2.956372
epoch:2862 loss:0.253507 accu:0.906250 gene_loss:2.955875
epoch:2863 loss:0.237423 accu:0.914062 gene_loss:3.147201
epoch:2864 loss:0.194855 accu:0.929688 gene_loss:3.657534
epoch:2865 loss:0.311786 accu:0.875000 gene_loss:3.361133
epoch:2866 loss:0.307796 accu:0.875000 gene_loss:4.181780
epoch:2867 loss:0.378388 accu:0.812500 gene_loss:5.970578
epoch:2868 loss:0.443815 accu:0.804688 gene_loss:3.916854
epoch:2869 loss:0.609292 accu:0.726562 gene_loss:4.657199
epoch:2870 loss:0.506199 accu:0.796875 gene_loss:3.177014
epoch:2871 loss:0.374115 accu:0.835938 gene_loss:3.848121
epoch:2872 loss:0.463005 accu:0.804688 gene_loss:3.312356
epoch:2873 loss:0.519991 accu:0.726562 gene_loss:2.793529
epoch:2874 los

In [33]:
172675753*32/(1024**3)



5.146138459444046

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [39]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         29029120  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 14320641  
Total params: 43,349,761
Trainable params: 29,025,536
Non-trainable params: 14,324,225
_________________________________________________________________


In [ ]:
gc.collect()